In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
tqdm.tqdm.pandas()

In [2]:
date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')

In [3]:
date_today_record = str(date_today).split(' ')[0].replace('-', '')
date_today_record

'20220224'

In [4]:
def day_modifier(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if len(x) == 1:
            return '0' + x
        else:
            return x

In [5]:
def landnum_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x).replace('외', '').replace(' ','')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [6]:
def data_prep(bdtype, tradetype, starting_year=2018):
    # bdtype is a string: one of 아파트, 연립다세대 or 오피스텔
    # tradetype is a string: one of 매매 or 전월세
    if (tradetype != '매매') & (tradetype != '전월세'):
        raise ValueError('두번째 변수는 매매 또는 전월세만 입력 가능')
    
    
    basedir = './국토교통부_실거래가_공개시스템/인천/{}/{}/'.format(bdtype, tradetype)
    filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]
    
    dfs_list = []
    for i, f in tqdm.tqdm_notebook(enumerate(filenames)):
        splitted_filename = f.split('실거래가_')       
        file_year = int(splitted_filename[-1][:4])
        
        '''if file_year < starting_year:
            continue'''
        
        try:
            df = pd.read_csv(basedir + f, encoding='euc-kr', header=15)
        except:
            try:
                df = pd.read_csv(basedir + f, encoding='utf-8', header=15)
            except:
                df = pd.read_csv(basedir + f, header=15)
        
        if '건물명' in df.columns:
            df.rename(columns={'건물명':'건물(단지)명'}, inplace=True)
        elif '단지명' in df.columns:
            df.rename(columns={'단지명':'건물(단지)명'}, inplace=True)
            
        if '대지권면적(㎡)' in df.columns:
            df = df.drop(columns=['대지권면적(㎡)'])
                
            
        if '해제사유발생일' in df.columns.tolist():
            df = df[df['해제사유발생일'].isna()]
            df = df.drop(columns=['해제사유발생일'])
            
        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    if tradetype == '전월세':
        concat_df = concat_df.rename(columns={'전월세구분':'거래구분', '보증금(만원)':'거래금액(만원)'})
        concat_df = concat_df.drop(columns=['월세(만원)'])
        concat_df = concat_df[concat_df['거래구분'] == '전세']
    elif tradetype == '매매':
        concat_df['거래구분'] = '매매'
        
    concat_df['번지'] = concat_df['번지'].apply(landnum_modifier)
    
    concat_df['계약년월'] = concat_df['계약년월'].astype('Int64')
    concat_df['계약일'] = concat_df['계약일'].astype('Int64')
    
    concat_df['계약년월'] = concat_df['계약년월'].apply(str)
    concat_df['계약일'] = concat_df['계약일'].apply(str)
    
    concat_df['계약날짜기준_건물연식'] = concat_df['계약년월'].apply(lambda x: int(x[:4])) - concat_df['건축년도']
    
    concat_df['계약일'] = concat_df['계약일'].apply(day_modifier)
    
    concat_df['계약날짜'] = concat_df['계약년월'].apply(lambda x: x[:4]) + '-' + concat_df['계약년월'].apply(lambda x: x[-2:])\
    + '-' + concat_df['계약일']
    
    concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'], format='%Y-%m-%d')
    
    concat_df['거래금액(만원)'] = concat_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
    concat_df['단가(만원/㎡)'] = concat_df['거래금액(만원)'] / concat_df['전용면적(㎡)']
    
    concat_df['지번주소'] = concat_df['시군구'] + ' ' + concat_df['번지']
        
    cols_to_drop = ['번지', '지번주소', '계약년월', '계약일', '도로명']
        
    concat_df = concat_df[['지번주소', '도로명'] + [col for col in concat_df.columns if col not in cols_to_drop]]
    
    date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')
    concat_df['건물연식'] = date_today.year - concat_df['건축년도']
    
    concat_df = concat_df.dropna(subset=['지번주소'])
    
    concat_df = concat_df[concat_df['층'] >= 0].reset_index(drop=True)
    
    return concat_df

In [7]:
apart_df = data_prep('아파트', '매매')
apart_df['부동산유형'] = '아파트'
print(apart_df.shape)
apart_df.head()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


0it [00:00, ?it/s]

(158894, 18)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,인천광역시 강화군 강화읍 갑곳리 165-1,갑룡길 35,인천광역시 강화군 강화읍 갑곳리,165,1,그랑드빌,84.89,15000,6,2001,-,-,매매,17,2018-03-06,176.699258,21,아파트
1,인천광역시 강화군 강화읍 갑곳리 165-1,갑룡길 35,인천광역시 강화군 강화읍 갑곳리,165,1,그랑드빌,114.86,20500,7,2001,-,-,매매,17,2018-04-12,178.478147,21,아파트
2,인천광역시 강화군 강화읍 갑곳리 165-1,갑룡길 35,인천광역시 강화군 강화읍 갑곳리,165,1,그랑드빌,84.89,17000,9,2001,-,-,매매,17,2018-09-13,200.259159,21,아파트
3,인천광역시 강화군 강화읍 갑곳리 165-1,갑룡길 35,인천광역시 강화군 강화읍 갑곳리,165,1,그랑드빌,84.89,18500,9,2001,-,-,매매,17,2018-11-06,217.929085,21,아파트
4,인천광역시 강화군 강화읍 갑곳리 165-1,갑룡길 35,인천광역시 강화군 강화읍 갑곳리,165,1,그랑드빌,84.89,18500,9,2001,-,-,매매,17,2018-12-05,217.929085,21,아파트


In [8]:
yunrip_df = data_prep('연립다세대', '매매')
yunrip_df['부동산유형'] = '연립다세대'
print(yunrip_df.shape)
yunrip_df.head()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


0it [00:00, ?it/s]

(92683, 18)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,인천광역시 강화군 강화읍 갑곳리 37-1,갑룡길 105-23,인천광역시 강화군 강화읍 갑곳리,37,1,갑룡라이너스A동,58.410,12000,2,2014.0,-,-,매매,4.0,2018-01-18,205.444273,8.0,연립다세대
1,인천광역시 강화군 강화읍 갑곳리 37-11,갑룡길 105-25,인천광역시 강화군 강화읍 갑곳리,37,11,갑룡라이너스B동C동,58.410,11000,1,2014.0,-,-,매매,4.0,2018-01-19,188.323917,8.0,연립다세대
2,인천광역시 강화군 강화읍 갑곳리 37-11,갑룡길 105-25,인천광역시 강화군 강화읍 갑곳리,37,11,갑룡라이너스B동C동,58.410,10500,1,2014.0,-,-,매매,4.0,2018-07-07,179.763739,8.0,연립다세대
3,인천광역시 강화군 강화읍 갑곳리 345-0,강화대로312번길 20-3,인천광역시 강화군 강화읍 갑곳리,345,0,강화,61.740,4600,3,1985.0,-,-,매매,33.0,2018-06-09,74.505993,37.0,연립다세대
4,인천광역시 강화군 강화읍 갑곳리 152-3,갑룡길5번길 11-1,인천광역시 강화군 강화읍 갑곳리,152,3,강화그린빌라(1동),49.015,6500,1,2002.0,-,-,매매,16.0,2018-06-14,132.612466,20.0,연립다세대


In [9]:
officetel_df = data_prep('오피스텔', '매매')
officetel_df['부동산유형'] = '오피스텔'
print(officetel_df.shape)
officetel_df.head()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


0it [00:00, ?it/s]

(22249, 18)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,인천광역시 강화군 강화읍 남산리 191-2,충렬사로22번길 16,인천광역시 강화군 강화읍 남산리,191,2,고려팰리스6차,84.97,16000,4,2018.0,-,-,매매,0.0,2018-09-05,188.301754,4.0,오피스텔
1,인천광역시 강화군 강화읍 남산리 191-2,충렬사로22번길 16,인천광역시 강화군 강화읍 남산리,191,2,고려팰리스6차,84.97,16000,3,2018.0,-,-,매매,0.0,2018-09-10,188.301754,4.0,오피스텔
2,인천광역시 강화군 강화읍 남산리 191-2,충렬사로22번길 16,인천광역시 강화군 강화읍 남산리,191,2,고려팰리스6차,84.97,16000,7,2018.0,-,-,매매,0.0,2018-09-13,188.301754,4.0,오피스텔
3,인천광역시 강화군 강화읍 남산리 191-2,충렬사로22번길 16,인천광역시 강화군 강화읍 남산리,191,2,고려팰리스6차,84.41,16000,3,2018.0,-,-,매매,0.0,2018-09-13,189.551001,4.0,오피스텔
4,인천광역시 강화군 강화읍 남산리 191-2,충렬사로22번길 16,인천광역시 강화군 강화읍 남산리,191,2,고려팰리스6차,84.41,16000,5,2018.0,-,-,매매,0.0,2018-09-13,189.551001,4.0,오피스텔


In [10]:
house_df = pd.concat([apart_df, yunrip_df, officetel_df]).sort_values(['시군구', '본번', '부번', '건축년도', '전용면적(㎡)', '계약날짜']).reset_index(drop=True)
house_df = house_df.drop(columns=['거래유형', '중개사소재지', '거래구분', '계약날짜기준_건물연식'])
del apart_df, yunrip_df, officetel_df
print(house_df.shape)
house_df.head()

(273826, 14)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,인천광역시 강화군 강화읍 갑곳리 15-0,갑룡길 105-13,인천광역시 강화군 강화읍 갑곳리,15,0,정흔하이빌,75.105,15500,2,2014.0,2018-06-21,206.377738,8.0,연립다세대
1,인천광역시 강화군 강화읍 갑곳리 15-0,갑룡길 105-13,인천광역시 강화군 강화읍 갑곳리,15,0,정흔하이빌,75.105,15000,2,2014.0,2021-09-24,199.720391,8.0,연립다세대
2,인천광역시 강화군 강화읍 갑곳리 26-1,갑룡길 95,인천광역시 강화군 강화읍 갑곳리,26,1,"승화래미향(1,2)동",64.350,11400,3,2009.0,2019-01-05,177.156177,13.0,연립다세대
3,인천광역시 강화군 강화읍 갑곳리 26-1,갑룡길 95,인천광역시 강화군 강화읍 갑곳리,26,1,"승화래미향(1,2)동",64.350,11100,1,2009.0,2021-05-13,172.494172,13.0,연립다세대
4,인천광역시 강화군 강화읍 갑곳리 26-1,갑룡길 95,인천광역시 강화군 강화읍 갑곳리,26,1,"승화래미향(1,2)동",64.350,11500,3,2009.0,2021-11-16,178.710179,13.0,연립다세대


In [11]:
'''land_specs_df = pd.read_csv('./prepped_data/land_specs_ver_4.csv')
print(land_specs_df.shape)
land_specs_df.head()'''

"land_specs_df = pd.read_csv('./prepped_data/land_specs_ver_4.csv')\nprint(land_specs_df.shape)\nland_specs_df.head()"

In [12]:
'''last_df = land_specs_df.drop_duplicates(subset=['지번주소'], keep='last').drop(columns=['년', '공시지가']).reset_index(drop=True)
print(last_df.shape)
last_df.head()'''

"last_df = land_specs_df.drop_duplicates(subset=['지번주소'], keep='last').drop(columns=['년', '공시지가']).reset_index(drop=True)\nprint(last_df.shape)\nlast_df.head()"

In [13]:
'''house_merge_df = house_df.merge(last_df, on=['지번주소']).reset_index(drop=True)
print(house_merge_df.shape)'''

"house_merge_df = house_df.merge(last_df, on=['지번주소']).reset_index(drop=True)\nprint(house_merge_df.shape)"

In [14]:
house_df['일괄계약'] = 'N'

In [15]:
#house_df = house_df.merge(last_df, on=['지번주소']).reset_index(drop=True)

In [16]:
'''print(house_df.shape)
house_df.head()'''

'print(house_df.shape)\nhouse_df.head()'

In [17]:
def identify_bulk_contract(df):
    df = df.copy()
    
    for addr in tqdm.tqdm_notebook(df['지번주소'].unique()):
        addr_df = df[df['지번주소'] == addr]
        
        for btyear in addr_df['건축년도'].unique():
            btyear_df = addr_df[addr_df['건축년도'] == btyear]
            
            if (btyear_df.shape[0] >= 10) & (btyear_df['계약날짜'].nunique() == 1):
                df.loc[btyear_df.index, '일괄계약'] = 'Y'

    return df

In [18]:
house_df['계약날짜'].describe()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                  273826
unique                   1515
top       2020-06-16 00:00:00
freq                      755
first     2018-01-01 00:00:00
last      2022-02-23 00:00:00
Name: 계약날짜, dtype: object

In [19]:
house_df = identify_bulk_contract(house_df)

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/23753 [00:00<?, ?it/s]

In [20]:
bulk_df = house_df[house_df['일괄계약'] == 'Y']
print(bulk_df.shape)
bulk_df.head()

(4686, 15)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형,일괄계약
649,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,67.94,14140,4,2018.0,2018-12-31,208.124816,4.0,연립다세대,Y
650,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,67.94,14140,3,2018.0,2018-12-31,208.124816,4.0,연립다세대,Y
651,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,67.94,14140,2,2018.0,2018-12-31,208.124816,4.0,연립다세대,Y
652,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,68.59,14275,5,2018.0,2018-12-31,208.120717,4.0,연립다세대,Y
653,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,69.09,14379,4,2018.0,2018-12-31,208.119844,4.0,연립다세대,Y


In [21]:
bulk_df['지번주소'].nunique()

246

In [22]:
%%time
bulk_df.to_excel('./공공주택찾기/인천/일괄계약전체_20180101_20220224.xlsx', index=False)

Wall time: 1.38 s
